# Collect Information

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from groq import Groq
import pickle
import os
import time

from textblob import TextBlob
from nltk.corpus import stopwords

In [2]:
'''
These are helper functions to add information to the database
'''

def get_number(course_name: str):
    '''
    This function takes in a course name and returns the course number

    Parameters
    ----------
    course_name : str
        The name of the course

    Returns
    -------
    The course number : str

    Example
    -------
    >>> get_number('CS 50 - Introduction to Computer Science')
    '50'
    '''
    _, number, *_ = course_name.split()

    return number[:number.index('-')]

def get_comments(df: pd.DataFrame, class_name: str, class_number: str, semester: str, year: int) -> list:
    '''
    This function takes in a class name, class number, semester, and year and returns the comments for that class

    Parameters
    ----------
    class_name : str
        The name of the class
    class_number : str
        The number of the class
    semester : str
        The semester of the class ('S' or 'F')
    year : int
        The year of the class

    Returns
    -------
    The comments for the class : list
    '''
    try:
        return eval(list(df[(df['department'].str.lower() == class_name.lower())\
                             & (df['class_number'].str.lower() == class_number.lower())
                             & (df['semester'].str.lower() == semester.lower())\
                                & (df['year'] == year)
                             ].comments.values)[0])

    except:
        return []
    
def get_id(df: pd.DataFrame, class_name: str, class_number: str, semester: str, year: int) -> str:
    '''
    This function takes in a class name, class number, semester, and year and returns the course id for that class

    Parameters
    ----------
    class_name : str
        The name of the class

    class_number : str
        The number of the class

    semester : str
        The semester of the class ('S' or 'F')

    year : int
        The year of the class

    Returns
    -------
    The course id for the class : str
    '''
    temp_df = df[(df['department'].str.lower() == class_name.lower()) & (df['class_number'].str.lower() == class_number.lower()) & (df['semester'].str.lower() == semester.lower()) & (df['year'] == year)]

    if len(temp_df):
        return temp_df['course_id'].values[0]
    
    return ""

def get_sentiment(text: str) -> float:
    '''
    This function takes in a text and returns the sentiment of the text

    Parameters
    ----------
    text : str
        The text to analyze

    Returns
    -------
    The sentiment of the text : float
    '''
    stop_words = set(stopwords.words('english'))
    filtered_comments = [word for word in text.split() if word.lower() not in stop_words]
    comment = " ".join(filtered_comments)
    blob = TextBlob(comment)
    sentiment = blob.sentiment.polarity
    

    return sentiment

def get_gemmy(st):
    st = st.lower()
    return (st.count('gem') + max(st.count("gemmy") - st.count("gem"), 0) - st.count("not a gem") - st.count("not gemmy")) / len(st)

def get_stars(st_lst):
    try:
        lst = eval(st_lst)
        total = 0
        for st in lst:
            try:
                if st[0] == '*':
                    total += 1
            except:
                pass

        return total / len(lst)
    except:
        return 0

In [8]:
def format_data(df: pd.DataFrame, start: int = 0) -> list[pd.DataFrame]:
    '''
    Takes in the dataframe as an output from collector.py and returns a list of dataframes

    These are:

    1. The courses dataframe
    2. The comments dataframe
    3. The instructors dataframe

    These are all connected by the column "course_id"
    '''

    # add the course number to the dataframe
    df['class_number'] = df.name.apply(get_number)

    # create a temprorary copy of the dataframe to store instructors
    temp_df = df.copy()

    # many classes have more than 1 professor
    # we want to combine all the professors into one class
    df = df.groupby(['department', 'class_number', 'semester', 'year']).agg({
        'name': 'first',
        'link': 'first',
        'hours': 'first',
        'students': 'first',
        'recommendations': 'first',
        'instructor_rating': 'first',
        'comments': 'first',
        'instructor': lambda x: ', '.join([str(item) for item in x]),
    }).reset_index()

    # we add course ids
    df['course_id'] = [thing + start for thing in list(range(len(df)))]

    # now, with temp_df, we want to create a dataframe for instructors
    # first, add the course_id to the dataframe
    temp_df['course_id'] = temp_df.apply(lambda x: get_id(df, x['department'], x['class_number'], x['semester'], x['year']), axis=1)
    temp_df = temp_df[['course_id', 'instructor', 'instructor_rating']]

    # here, we want to remove any ratings that are greater than 5
    # this means that an error had occured in the collection process, so we ignore it
    temp_df['instructor_rating'] = np.where((temp_df['instructor_rating'] > 5) | (temp_df['instructor_rating'] < 0), np.nan, temp_df['instructor_rating'])
    df['recommendations'] = np.where((df['recommendations'] > 5) | (df['recommendations'] < 0), np.nan, df['recommendations'])

    # we also want to create a dataframe specifically for comments
    # this may help display on the frontend should we turn this project into a web application
    comments_df = {
        'course_id' : [],
        'comment' : [],
        'department' : [],
        'class_number' : [],
    }

    # we add the comments to the comments dataframe
    for course_id, comments, department, class_number in df[['course_id', 'comments', 'department', 'class_number']].values:
        for c in eval(comments):
            comments_df['course_id'].append(course_id)
            comments_df['comment'].append(c)
            comments_df['department'].append(department)
            comments_df['class_number'].append(class_number)

    comments_df = pd.DataFrame(comments_df)

    average_sentiments = []

    for i, course_comments in enumerate(df.comments):
        all_comments = eval(course_comments)
        sentiments = [get_sentiment(comment) for comment in all_comments]
        average_sentiments.append(np.mean(sentiments) if len(sentiments) >= 1 else np.nan)

    df['avg_sentiment'] = average_sentiments

    # we add the sentiment to the comments dataframe
    comments_df['sentiment'] = comments_df.comment.apply(get_sentiment)

    df['frac_stars'] = df.comments.apply(get_stars) * 1500
    df['frac_gems'] = df.comments.apply(get_gemmy) * 1500

    df['easy_name'] = df['department'] + ' ' + df['class_number']

    return [df.drop('instructor_rating', axis=1), comments_df, temp_df]

In [9]:
df_1 = pd.read_csv('qreport_app/raw_data/output_2021_2022.csv')
df_2 = pd.read_csv('qreport_app/raw_data/output_2022_2023.csv')
df_3 = pd.read_csv('qreport_app/raw_data/output_2023_2024.csv')

df_1, comments_df_1, temp_df_1 = format_data(df_1, start = len(df_2) + len(df_3))
df_2, comments_df_2, temp_df_2 = format_data(df_2, start = len(df_3))
df_3, comments_df_3, temp_df_3 = format_data(df_3)

df_1.to_csv('qreport_app/working_data/courses_2021_2022.csv', index=False)
comments_df_1.to_csv('qreport_app/working_data/comments_2021_2022.csv', index=False)
temp_df_1.to_csv('qreport_app/working_data/instructors_2021_2022.csv', index=False)

df_2.to_csv('qreport_app/working_data/courses_2022_2023.csv', index=False)
comments_df_2.to_csv('qreport_app/working_data/comments_2022_2023.csv', index=False)
temp_df_2.to_csv('qreport_app/working_data/instructors_2022_2023.csv', index=False)

df_3.to_csv('qreport_app/working_data/courses_2023_2024.csv', index=False)
comments_df_3.to_csv('qreport_app/working_data/comments_2023_2024.csv', index=False)
temp_df_3.to_csv('qreport_app/working_data/instructors_2023_2024.csv', index=False)